In [ ]:
# Import functions
import numpy as np
import matplotlib.pyplot as p
%matplotlib inline

import eigensource.add_noise as add_noise
import eigencurves
import eigenmaps
import kmeans
#import mapPCA
import bin_eigenspectra

from importlib import import_module
planet_name = 'HD189733b'
system = import_module('data.planet.{}'.format(planet_name))

### Import spectra and generate map

In [ ]:
# ...

### Generate lightcurve using STARRY

In [ ]:
# from gen_lightcurves import prep_map1, create_lightcurves_with_starry
# lam, spaxels = prep_map1()
# time, lam, dlam, lcurves = create_lightcurves_with_starry(lam, spaxels)

Plot map

In [ ]:
# Load lightcurve
stuff = np.load("data/input_lightcurves/eclipse_lightcurve_test1.npz")

# Parse File
lightcurve = stuff["lightcurve"]
wl = stuff["wl"]
dwl = stuff["dwl"]
time = stuff["time"]

# Make Plot
fig, ax = p.subplots(1, figsize=(14, 5))
ax.set_xlabel('Time [days]')
ax.set_ylabel('Relative Flux')
for i in range(len(wl)):
    lc = lightcurve[:,i] - np.min(lightcurve[:,i])
    ax.plot(time, lc+1, c = "C%i" %(i%9), label = r"%.2f $\mu$m" %(wl[i]))
ax.legend(fontsize = 16, ncol = 2)
p.show()

### Add Noise

In [ ]:
inputLC3D = add_noise.get_lc()
noiseDict = add_noise.add_noise(inputLC3D)

### Fit eigencurves to lightcurve
Actually this is done with `run_higher_sph_harm.py` with results saved in `data/sph_harmonic_coefficients/`

In [ ]:
# import importlib
# importlib.reload(eigencurves)

#spherearray = eigencurves.eigencurves(noiseDict,plot=True)
# spherearray is an array of wavelength x SH coefficents

## Show the original map

In [ ]:
origData = np.load("data/maps/mystery_map1.npz")
import gen_lightcurves
import healpy

In [ ]:
lammin1 = 2.41; lammax1 = 3.98; dlam1 = 0.18
spaxels = origData["spaxels"]
lam = origData["wl"]
lamlo, dlamlo = gen_lightcurves.construct_lam(lammin1, lammax1, dlam=dlam1)
Nlamlo = len(lamlo)

# Set HealPy pixel numbers
Npix = spaxels.shape[0]

# Define empty 2d array for spaxels
spec2d = np.zeros((Npix, Nlamlo))

# Loop over pixels filling with spectra
for i in range(Npix):
    # Degrade the spectra to lower resolution
    spec2d[i,:] = gen_lightcurves.downbin_spec(spaxels[i, :], lam, lamlo, dlam = dlamlo)

def show_orig_map(ind=0):
    healpy.mollview(spec2d[:,ind], title=r"%0.2f $\mu$m" %lamlo[ind])
    p.show()

In [ ]:
show_orig_map(ind=5)

## Show retrieved map

In [ ]:
def retrieved_map(ngroups=4,degree=3,waveInd=3):
    tmp = np.load("data/sph_harmonic_coefficients/spherearray_{}.npz".format(degree))
    sphereHArray = tmp['arr_0']
    
    londim = 100
    latdim = 100
    samples = np.array([sphereHArray]) # output from eigencurves
    
    wavelengths, lats, lons, maps = eigenmaps.generate_maps(samples,
                                                            N_lon=londim, N_lat=latdim)
    waves = wavelengths[0]
    
    map_day = maps[0][waveInd][:,londim//4:-londim//4]
    extent = np.array([np.min(lons),np.max(lons),np.min(lats),np.max(lats)])/2./np.pi*180
    ax = p.imshow(map_day, extent=extent)
    cbar = p.colorbar(ax, ticks=np.arange(ngroups))
    cbar.set_label('Brightness')
    p.ylabel('Latitude')
    p.xlabel('Longitude')
    p.title('Retrieved group map, n={}, {:.2f}$\mu$m'.format(degree,waves[waveInd]))
    p.show()

In [ ]:
retrieved_map(waveInd=5,degree=5)

## Check lightcurves

In [ ]:
import sys
import lightcurves_sh

if (sys.version_info > (3, 0)):
    from importlib import reload

reload(eigencurves)
reload(lightcurves_sh)

In [ ]:
def check_lightcurve(degree=3,waveInd=0):
    """ Check the lightcurve """
    tmp = np.load("data/sph_harmonic_coefficients/spherearray_{}.npz".format(degree))
    coeff = tmp['arr_0']
    t = noiseDict['time (days)']
    
    sp_params = lightcurves_sh.spider_model(degree=degree,t0=-2.21857567/2.)
    sp_params.sph = list(coeff[waveInd,1:])
    lc = sp_params.lightcurve(t)
    fig, (ax0, ax1) = p.subplots(2)
    resid = noiseDict['flux (ppm)'][:,waveInd]- lc * 1e6
    ax0.errorbar(t,noiseDict['flux (ppm)'][:,waveInd],fmt='o',
                 yerr=noiseDict['flux err (ppm)'][:,waveInd])
    ax0.plot(t,lc * 1e6,label='fit')
    ax1.errorbar(t,resid,yerr=noiseDict['flux err (ppm)'][:,waveInd],
                 fmt='o')
    tmp.close()

In [ ]:
check_lightcurve(waveInd=5,degree=5)

## Troubleshooting fits stuff

Make a dictionary for checking with just two wavelengths

In [ ]:
# checkDict = {}
# checkDict['flux err (ppm)'] = noiseDict['flux err (ppm)'][:,5:7]
# checkDict['time (days)'] = noiseDict['time (days)']
# checkDict['wavelength (um)'] = noiseDict['wavelength (um)'][5:7]
# checkDict['flux (ppm)'] = noiseDict['flux (ppm)'][:,5:7]

In [ ]:
# tmp = np.load("data/sph_harmonic_coefficients/spherearray_{}.npz".format(3))
# coeff = tmp['arr_0']
# coeff[5,:]


In [ ]:
# spherearray = eigencurves.eigencurves(checkDict,plot=True,
#                                       sph_harm_degree=3)

In [ ]:
# check_lightcurve(degree=3,waveInd=0)

In [ ]:
# spherearray = eigencurves.eigencurves(noiseDict,plot=True,
#                                       sph_harm_degree=3)
# #spherearray is an array of wavelength x SH coefficents

## Get Eigenspectra and Map

In [ ]:
def spec_and_map(ngroups=4,degree=3,specOrder=None):
    tmp = np.load("data/sph_harmonic_coefficients/spherearray_{}.npz".format(degree))
    sphereHArray = tmp['arr_0']
    
    londim = 100
    latdim = 100
    samples = np.array([sphereHArray]) # output from eigencurves
    
    eigenspectra_draws = []
    kgroup_draws = []
    
    wavelengths, lats, lons, maps = eigenmaps.generate_maps(samples,
                                                            N_lon=londim, N_lat=latdim)
    waves = wavelengths[0]
    
    for draw, map_ in zip(samples, maps):
        kgroups = kmeans.kmeans(map_, ngroups)
        
        eigenspectra = bin_eigenspectra.bin_eigenspectra(map_, kgroups)
        
        eigenspectra_draws.append(eigenspectra)
        kgroup_draws.append(kgroups)
        
    eigenspectra = np.mean(eigenspectra_draws, axis=0)
    eigenerrs = np.std(eigenspectra_draws, axis=0)
    
    kgroups = np.mean(kgroup_draws, axis=0)
    
    fig, (ax0, ax1) = p.subplots(1,2,figsize=(12,4))
    for ind,spec, err in zip(range(ngroups), eigenspectra, eigenerrs):
        ax0.errorbar(waves, spec, err, marker='o',
                  color=p.cm.viridis(float(ind)/float(ngroups-1)))
    ax0.set_xlabel('Wavelength (micron)')
    ax0.set_ylabel('Fp/Fs (ppm)')
    ax0.set_title('Eigenspectra from light-curve fit, n={}'.format(degree))
    
    kgroups_day = kgroups[:,londim//4:-londim//4]
    extent = np.array([np.min(lons),np.max(lons),np.min(lats),np.max(lats)])/2./np.pi*180
    outAx = ax1.imshow(kgroups_day, extent=extent)
    cbar = p.colorbar(outAx, ticks=np.arange(ngroups))
    cbar.set_label('# Group')
    ax1.set_ylabel('Latitude')
    ax1.set_xlabel('Longitude')
    ax1.set_title('Retrieved group map, n={}'.format(degree))
    
    fig.savefig('plots/eigenmap_and_spec/eigenmap_and_spec_n_{}.pdf'.format(degree))
    
    fig.show()


### Show original 4 spectra

In [ ]:
uniqueSpec, uniqueMap = np.unique(spec2d,axis=0,return_inverse=True)
nUniqueSpec = uniqueSpec.shape[0]
groupSpec2D = np.zeros(spec2d.shape[0])
fig, ax0 = p.subplots()
for ind,oneSpec in enumerate(uniqueSpec):
    color=p.cm.viridis(float(ind)/float(nUniqueSpec-1))
    ax0.plot(noiseDict['wavelength (um)'],oneSpec,color=color,label="Spec {}".format(ind))
ax0.legend()
ax0.set_xlabel('Wavelength ($\mu$m)')
ax0.set_ylabel('Relative Flux')
fig.savefig('plots/original_maps/orig_quadrant_spec.pdf')
plotObj = healpy.mollview(uniqueMap,title='Spectral map',unit='Spectrum #')
p.savefig('plots/original_maps/orig_quadrant_map.pdf')

In [ ]:
fig, ax = p.subplots()
healpy.mollview(uniqueMap,title='Spectral map',hold=True,unit='Spectrum #')

In [ ]:
spec_and_map(ngroups=3,degree=oneDeg)

In [ ]:
for oneDeg in np.arange(2,6+1):
    spec_and_map(ngroups=4,degree=oneDeg)